## Hands-On with Delta Lake
- Explore practical aspects of working with Delta Lake in Databricks.
- Walk through key tasks:
  - Creating Delta Lake tables
  - Inserting data
  - Updating tables with new data
  - Examining the directory structure
- Gain a clear understanding through hands-on examples.

**Databricks Table Organization**
- Tables are organized in databases under a catalog.
- For these exercises, we’ll use the hive_metastore catalog.
- hive_metastore is the default catalog available in all Databricks workspaces.

In [0]:
USE CATALOG hive_metastore

### Creating Delta Lake Tables

- Similar to standard SQL using CREATE TABLE.
- Define table name and schema (columns with data types).
- Example: Create an empty table named product_inventory.

In [0]:
DROP TABLE IF EXISTS product_inventory;

CREATE TABLE product_inventory (
  product_id INT,
  product_name STRING,
  category STRING,
  price DOUBLE,
  quantity INT
)
USING DELTA;

After creating the Delta Lake table `product_inventory` using the SQL script, you can view it through the **Catalog Explorer** by clicking the **Catalog** tab in the left sidebar of your Databricks workspace.

### Inserting Data

In [0]:
INSERT INTO product_inventory (product_id, product_name, category, price, quantity)
VALUES 
(1, 'Running Shoes', 'Footwear', 59.99, 150),
(2, 'Air Fryer', 'Appliances', 139.95, 35),
(3, 'Action Figure', 'Toys', 22.50, 80),
(4, 'Wireless Mouse', 'Electronics', 49.99, 90);

To access and verify the inserted data, simply query the table through the standard
SQL SELECT statement.

In [0]:
SELECT * FROM product_inventory

### Exploring Table Directory

- Executing transactional operations created two data files in the table directory.
- Use the `DESCRIBE DETAIL` command to validate and inspect table metadata.
- This command displays key information, including:
  - `numFiles`: shows the number of data files in the current table version.
  - `location`: displays the table’s location

In [0]:
DESCRIBE DETAIL product_inventory

To examine the table's file structure in more detail, you can use the `%fs` magic command to browse the contents of the directory.

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory'

- The command confirms that the table directory contains two data files in Parquet format.
- It also shows the _delta_log subdirectory, which holds the transaction log files for the table.

### Updating Delta Lake Tables
let's look at a scenario where you need to modify the price of a product.

In [0]:
UPDATE product_inventory
SET price = price * 1.10
WHERE product_id = 3

After the update operation, examining the table directory reveals an important detail — a new data file has been added.

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory'

- When updates occur, Delta Lake doesn’t overwrite existing files; instead, it creates new updated copies. 
- It then uses the transaction log to track which files are valid in the current table version. 

Run the **DESCRIBE DETAIL** command again to verify this behavior.

In [0]:
DESCRIBE DETAIL product_inventory

#### What are deletion vectors?
- Deletion vectors are a storage optimization feature available in Delta Lake.
- Default, deleting a single row requires rewriting the entire Parquet file containing that row.
- When deletion vectors are enabled:
  - `DELETE`, `UPDATE`, and `MERGE` operations mark rows as removed or updated without rewriting the file.
  - Reads automatically apply the deletion vectors to present the correct current table state.
> Starting from Databricks Runtime Version 14

### Exploring Table History
- Delta Lake’s transaction log records the complete history of all table changes.
- You can view this history using the **DESCRIBE HISTORY** command.

In [0]:
DESCRIBE HISTORY product_inventory

> In **Databricks Runtime 10.4 LTS** and above, auto compaction and optimized writes are always enabled for `MERGE`, `UPDATE`, and `DELETE` operations. 

> You cannot disable this functionality.

The transaction log is located under the `_delta_log` folder in the table directory. You can navigate to this folder using the `%fs` magic command

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory/_delta_log'

In [0]:
-- %fs head dbfs:/user/hive/warehouse/product_inventory/_delta_log/00000000000000000003.json

### Exploring Delta Time Travel

**Exploring Delta Time Travel**

* Time Travel lets you explore the historical evolution of data in Delta Lake tables.
* Delta Lake automatically versions tables, capturing every change as a new version.
* Each version reflects the table’s state at a specific point in time, creating a full audit trail.
* Use the **`DESCRIBE HISTORY`** command in SQL to view past versions.
  * Shows operation details, timestamps, operation types (insert, update, delete), and related metadata.


In [0]:
DESCRIBE HISTORY product_inventory

#### Querying Older Versions

- Delta Lake automatically assigns each table operation a unique version number and timestamp.
- You can query previous table states in two ways:



**Querying by Timestamp**
  - Retrieve the table as it existed at a specific point in time.
  - Use TIMESTAMP AS OF in your query:
      
      `SELECT * FROM <table_name> TIMESTAMP AS OF <timestamp>`

  

In [0]:
SELECT * FROM product_inventory 
TIMESTAMP AS OF "2025-07-01T15:36:01.000+00:00"

**Querying by Version Number**
  - Access the table using its version number associated with each operation.
      
      `SELECT * FROM <table_name> VERSION AS OF <version>`

In [0]:
SELECT * FROM product_inventory 
VERSION AS OF 1

In [0]:
SELECT * FROM product_inventory@v1

### Rolling Back to Previous Versions

* Delta Lake Time Travel is ideal for undoing unintended data changes or bad writes.
* You can restore a table to an earlier state using the `RESTORE TABLE` command.
* Roll back by specifying either a timestamp or a version number:

  `RESTORE TABLE <table_name> TO TIMESTAMP AS OF <timestamp>`
  
  `RESTORE TABLE <table_name> TO VERSION AS OF <version>`

* Example: Recover deleted data from the `product_inventory` table by restoring it to a previous version.


In [0]:
DELETE FROM product_inventory

In [0]:
SELECT * FROM product_inventory

In [0]:
DESCRIBE HISTORY product_inventory

To revert the table to the state it was in before the deletion, specifically version 3, you can run the `**RESTORE TABLE**` command.

In [0]:
RESTORE TABLE product_inventory TO VERSION AS OF 3

In [0]:
SELECT * FROM product_inventory

In [0]:
DESCRIBE HISTORY product_inventory

### Optimizing Delta Lake Tables
- Delta Lake offers a feature to improve performance by compacting small files into larger ones.
- Compaction significantly speeds up read queries on the table.
- Use the `OPTIMIZE <table_name>` command to trigger this process.
- Ideal for tables with many small files created by frequent write operations.
- Example: ten small files consolidated into two larger files after optimization.

Table optimization boosts overall performance by reducing file management overhead and improving the efficiency of data retrieval operations.

**Z-Order Indexing**
* Z-Order indexing is an enhancement of the `OPTIMIZE` command.
* It reorganizes and co-locates column data within files to improve query performance.
* To apply Z-Order indexing, use `ZORDER BY` with the column names:

  `OPTIMIZE <table_name>`<br>
  `ZORDER BY (<column_names>)`



> Delta Lake's **OPTIMIZE** command, combined with Z-Order indexing, provides a powerful way to boost table performance by compacting small files and efficiently organizing column data to accelerate read queries.

In [0]:
OPTIMIZE product_inventory
ZORDER BY product_id

In [0]:
DESCRIBE DETAIL product_inventory
-- DESCRIBE EXTENDED product_inventory

In [0]:
DESCRIBE HISTORY product_inventory

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory'

### Vacuuming
- Vacuuming in Delta Lake efficiently manages unused data files in a table.
- Obsolete files can accumulate over time due to uncommitted changes or outdated table states.
- The `VACUUM` command removes these files to save storage space and reduce costs.
- You set a retention period, which defines how old files must be before they are eligible for deletion.
- By **default**, the retention period is `7 days` to avoid deleting files still needed by active operations.

> **Important consideration**: After vacuuming, you can no longer time-travel to table versions older than the retention period because their files are permanently deleted. <br>
Always choose the retention threshold carefully, aligning with your data retention and compliance policies.

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory'

In [0]:
VACUUM product_inventory RETAIN 0 HOURS

- The command throws an exception if the retention period is set lower than the default 7 days.
- As a workaround for demonstration purposes, you can temporarily disable the retention duration check in Delta Lake.
> **Important**: This approach is not recommended for production because it can lead to data integrity issues.

In [0]:
SET spark.databricks.delta.retentionDurationCheck.enabled = false

In [0]:
VACUUM product_inventory RETAIN 0 HOURS

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory'

In [0]:
DESCRIBE HISTORY product_inventory

In [0]:
SELECT * FROM product_inventory@v1

### Dropping Delta Lake Tables

In [0]:
DROP TABLE product_inventory

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/product_inventory'